# Initialize Truveta SDK

In [1]:
# These are some commonly used R Packages.  
# The arrow package makes loading data with spark faster. 
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.notebook.study)
library(sparklyr)
library(ggplot2)
library(reshape2)

In [2]:
print("load snapshot")
con <- create_connection()
study <- get_study(con)
#print(study)
population_id = "ps-3ormi7swwukuhhu6kcqrqw4mue"
population <- get_population(con, study, title = "PancreaticMainPop")
snapshot <- get_latest_snapshot(con, population)
snapshot
# get list of tables from the snapshot
tables <- get_tables(con, snapshot)
tables

In [3]:
#Get your working directory
# use fs = true when reading and writing files locally

output_path_local <- get_output_path(con, study, fs = TRUE)
output_path_local

In [4]:
display_plot<- \(x, dpi = "screen", ...) {
    file <- tempfile()
    # dump as PNG
    ggplot2::ggsave(file, device = "png", plot = x, dpi = dpi, ...)
    # load as base64
    uri <- base64enc::dataURI(file = file, mime = "image/png")
    unlink(file)
    # display as HTML
    displayHTML(paste0('<img src="', uri, '">'))
}

In [10]:
#Read from file to Notebook 
t1 <- paste(output_path_local, "/weight_data.csv.r", sep = "")

# use read.csv to read file into a R dataframe
weight_data <- read.csv(t1)
#nrow(AdvChemoMed_tb)
##display_df(cases_control,10)

In [14]:
# Convert R DataFrame to Spark DataFrame
temp_data <- as.DataFrame(weight_data)
createOrReplaceTempView(temp_data, "weight_data_sql")

In [55]:
# Build path
file_to_write <- paste(output_path_local, "/cases_control_allvar_new.csv.r", sep = "")

# use write.csv to write your file
write.csv(cases_control_allvar_new, file_to_write, row.names = FALSE)

### BMI derivation from DEfBodyMass

In [7]:
# Currently using this
sql <-
"
WITH temp as (
    SELECT PersonId, EffectiveDateTime, NormalizedValueUOMConceptId, NormalizedValueNumeric
    FROM SearchResult_defTrBodyMassIndexBmi 
        WHERE (NormalizedValueNumeric IS NOT NULL AND NormalizedValueNumeric >= 12 AND NormalizedValueNumeric <= 90)
        AND (EffectiveDateTime IS NOT NULL AND year(EffectiveDateTime) >= 2018) 
        AND NormalizedValueUOMConceptId IS NOT NULL
),

temp1 as (
  SELECT t.*, c.ConceptName as UnitOfMeasurement
  from temp t join Concept c on t.NormalizedValueUOMConceptId = c.ConceptId
),

temp2 AS (
    SELECT  t.PersonId,  t.EffectiveDateTime AS BMIDate, t.NormalizedValueNumeric AS BMI,t.UnitOfMeasurement, 
      -- Calculate absolute difference in days
      ABS(DATEDIFF(t.EffectiveDateTime, p.PanDt)) AS days
    FROM data p 
    INNER JOIN temp1 t ON t.PersonId = p.PersonId
    WHERE t.EffectiveDateTime BETWEEN DATEADD(day, -360, p.PanDt) AND DATEADD(day, 360, p.PanDt)
),

temp3 as (
 SELECT PersonId,BMI, BMIDate,
  CASE 
    WHEN BMI < 18.5 THEN 'Underweight'
    WHEN BMI >= 18.5 AND BMI < 25 THEN 'Normal'
    WHEN BMI >= 25 AND BMI < 30 THEN 'Overweight'
    WHEN BMI >= 30 AND BMI < 35 THEN 'Obese(Class I)'
    WHEN BMI >= 35 AND BMI < 40 THEN 'Obese(Class II)' 
    WHEN BMI >= 40 THEN 'Obese(Class III)'
  END AS BMI_Group,
  ROW_NUMBER() OVER (PARTITION BY PersonId ORDER BY days) AS rn
 FROM temp2
)

SELECT PersonId,BMI, BMIDate,BMI_Group FROM temp3 WHERE rn=1
"
bmi_temp <- load_sql_table(con, snapshot, sql, view_name='bmi_temp',output_mode = "sparklyr") 

In [9]:
bmi_temp <- collect(bmi_temp)
summary(bmi_temp)

### BMI from Observation Table

In [11]:
#BMI codes

bmi_loinc_codes = codeset(con,snapshot,"LOINC", 'selfAndDescendants', "39156-5")
bmi_snomed_codes = codeset(con,snapshot,"SNOMED CT", 'selfAndDescendants', "60621009")
bmi_code = rbind(bmi_loinc_codes,bmi_snomed_codes)

create_view(bmi_code,"bmi_code")

truvetaCode = codeset(con,snapshot,"Truveta", 'self',"1065712","1065714")
create_view(truvetaCode,"truvetaCode")

ht_code = codeset(con,snapshot,"Truveta", 'self', "779602")

wt_code = codeset(con,snapshot,"Truveta", 'self',"773683", "800318")
create_view(wt_code,"wt_code")

bmi_obv_code = codeset(con,snapshot,"Truveta", 'self', "809891")

bmi_all = rbind(bmi_loinc_codes, bmi_snomed_codes, bmi_obv_code)
create_view(bmi_all,"bmi_all")

In [30]:
# BMI value from the SearchResult_def4VitalSigns table
sql <- "

-- select all necessary required variables and keep only values with valid status, non null date and time

WITH temp as (
    SELECT PersonId, EffectiveDateTime, NormalizedValueNumeric, NormalizedValueConceptId,NormalizedValueUOMConceptId,
    explode_outer(CodeConceptId) as Code
    FROM SearchResult_def4VitalSigns 
        WHERE (StatusConceptId IS NOT NULL AND StatusConceptId NOT IN (SELECT ConceptId FROM truvetaCode)) AND
        NormalizedValueNumeric IS NOT NULL AND EffectiveDateTime IS NOT NULL
),

-- Join with Concept to get the name of ConceptId and Unit

temp2 as (
  SELECT *, Code.Id as CodeConceptId
   FROM temp
),

-- Join with Concept to get the name of  Unit

temp3 as (
  SELECT t.*, c.ConceptName as UnitOfMeasurement
  from temp2 t join Concept c on t.NormalizedValueUOMConceptId = c.ConceptId
  WHERE t.NormalizedValueNumeric >= 0 AND year(t.EffectiveDateTime) >= 2018
),

-- Since only bmi values within baseline are required add the time frame

temp4 as (
    SELECT t.* from temp3 t INNER JOIN data p
    ON t.PersonId = p.PersonId
    WHERE t.EffectiveDateTime BETWEEN DATEADD(day, -360, p.PanDt) AND DATEADD(day,360,p.PanDt)
),

-- Consider only distinct values from all columns and cchoose bmi_codes

temp5 as (
  SELECT DISTINCT 
       p.PersonId, p.EffectiveDateTime as BMIDate, round(p.NormalizedValueNumeric,2) as BMI, p.UnitOfMeasurement, p.CodeConceptId, c.ConceptName
    FROM temp4 p join bmi_all c on p.CodeConceptId = c.ConceptId
),

temp6 as (
    SELECT *,
    ROW_NUMBER() OVER (PARTITION BY PersonId ORDER BY BMIDate DESC) AS rn
    FROM temp5 
    WHERE BMI >= 12 AND BMI <= 90
)

SELECT * FROM temp6 WHERE rn=1

"
bmi_vs_temp <- load_sql_table(con,snapshot,sql, view_name = 'bmi_vs_temp', output_mode = 'sparklyr')
#display_df(bmi_vs_temp)

In [31]:
# Check BMI count

sql_count <- "
SELECT count (*) as count, mean(BMI), median(BMI), min(BMI), max(BMI) from bmi_vs_temp
"
display_df(load_sql_table(con, snapshot, sql_count,output_mode = "sparklyr"))

In [27]:
# Get the updated Vital signs table with all codes

sql <- "

-- select all necessary required variables and keep only values with valid status, non null date and time

WITH temp as (
    SELECT PersonId, EncounterId, EffectiveDateTime, NormalizedValueUOMConceptId, NormalizedValueNumeric, NormalizedValueConceptId,
    explode_outer(CodeConceptId) as Code
    FROM SearchResult_def4VitalSigns 
        WHERE (StatusConceptId IS NOT NULL AND StatusConceptId NOT IN (SELECT ConceptId FROM truvetaCode)) AND
        NormalizedValueNumeric IS NOT NULL AND
        EffectiveDateTime IS NOT NULL AND NormalizedValueUOMConceptId IS NOT NULL
),

-- Join with Concept to get the name of ConceptId and Unit

temp2 as (
  SELECT *, Code.Id as CodeConceptId
   FROM temp
),

temp3 as (
    SELECT p.*, c.ConceptName as VitalSign
    from temp2 p join Concept c on p.CodeConceptId = c.ConceptId
),

-- Join with Concept to get the name of  Unit

temp4 as (
  SELECT t.*, c.ConceptName as UnitOfMeasurement
  from temp3 t join Concept c on t.NormalizedValueUOMConceptId = c.ConceptId
  WHERE t.NormalizedValueNumeric >= 0 AND year(t.EffectiveDateTime) >= 2018
),

-- Since only lab values within baseline are required add the time frame

temp5 as (
    SELECT t.*, p.PanDt from temp4 t INNER JOIN data p
    ON t.PersonId = p.PersonId
),

-- Consider only distinct values from all columns and create a update unitofmeasurement column

temp6 as (
  SELECT DISTINCT 
       PersonId, VitalSign, CodeConceptId, EncounterId, EffectiveDateTime, NormalizedValueNumeric, NormalizedValueUOMConceptId,
  CASE 
   WHEN UnitOfMeasurement IN (
     'No Information', 
     'Field has not been mapped', 
     'Invalid', 
     'Field is not present in source'
   ) THEN 'Unknown'
   ELSE UnitOfMeasurement
 END AS UnitOfMeasurement
 FROM temp5
)

SELECT * FROM temp6
"
vital_sign_data <- load_sql_table(con,snapshot,sql, view_name = 'vital_sign_data', output_mode = 'sparklyr')
display_df(vital_sign_data)

#Weight from VS


In [35]:
 # Weight join with weight code
 sql <- "
      SELECT vs.PersonId, vs.VitalSign, vs.EncounterId, vs.EffectiveDateTime as WeightDate,
       vs.NormalizedValueNumeric as Weight, vs.UnitOfMeasurement as WeightUnit
      FROM vital_sign_data vs INNER JOIN wt_code w ON vs.CodeConceptId = w.ConceptId
  "
weight_data <- load_sql_table(con, snapshot, sql, view_name = 'weight_data', output_mode = "sparklyr")

In [36]:
# Check Weight count

sql_count <- "
SELECT count (*) as count, mean(Weight), median(Weight), min(Weight), max(Weight) from weight_data
"
display_df(load_sql_table(con, snapshot, sql_count,output_mode = "sparklyr"))

In [37]:
# Take non-weight unit out
weight_data <- weight_data |> 
    filter(WeightUnit != 'per liter' & 
    WeightUnit != 'per meter'&
    WeightUnit != 'billion per liter'&
    WeightUnit != 'centimeter'&
    WeightUnit != 'degree'&      
    WeightUnit != 'foot (US)'  &
    WeightUnit != 'heart beats per minute' & 
    WeightUnit != 'inches'   &
    WeightUnit != 'liter'   &
    WeightUnit != 'liter per minute'   &
    WeightUnit != 'lumen'   &
    WeightUnit != 'meter'   &
    WeightUnit != 'millimeter mercury column'   &
    WeightUnit != 'millivolt'   &
    WeightUnit != 'minute'   &
    WeightUnit != 'per hour'   &
    WeightUnit != 'per meter'   &
    WeightUnit != 'per liter'   &
    WeightUnit != 'percent'   &
    WeightUnit != 'second'   &
    WeightUnit != 'week'  &
    WeightUnit != 'Each'  &    
    WeightUnit != 'Inches'  &  
    WeightUnit != 'inch (international)'  &     
    WeightUnit != 'each'   
    )
    
    weight_data_clean <- weight_data %>%
    group_by(WeightUnit) %>%
    summarize(n = n(), 
             mean = mean(Weight, na.rm = TRUE),
             min = min(Weight, na.rm = TRUE),
             max = max(Weight, na.rm = TRUE),
             median = median(Weight, na.rm = TRUE)) 

display_df(weight_data_clean,100)


In [39]:
lbs_ll <-  90
lbs_ul <- 700
weight_data <- collect(weight_data)

weight_data <- weight_data %>%
    mutate(UOM_assumed = case_when(
               WeightUnit %in% 
                 c("pound (US and British)", "pound (US)", "pound (apothecary)") ~ "pound", # when pounds unit then pound
               WeightUnit != "unknown" ~ WeightUnit, 
               Weight >lbs_ll*453.6 & Weight <= lbs_ul*453.6 ~ "gram",   # when in range Gram
               Weight > lbs_ll*16 & Weight <= lbs_ul*16 ~ "ounce (avoirdupois)", # when value in range Ounce
               Weight <125 ~ "kilogram", # less than 125 = KG
           ),
        pounds = case_when(
            UOM_assumed %in% c("pound", "pound (US and British)", "pound (US)", 
               "pound (apothecary)") ~ Weight, # create pounds calculation based on groupings.
            UOM_assumed == "ounce (avoirdupois)" ~ Weight/16,
            UOM_assumed == "kilogram" ~ Weight *2.205,
            UOM_assumed == "gram" ~ Weight/453.6)
          ) %>%
     mutate(pounds = ifelse(pounds < lbs_ll | pounds > lbs_ul, NA, pounds), # if pounds outside pluasible range then NA
            kg = pounds/2.205) # calculate kg from pounds


In [45]:
weight_data <- weight_data  %>%
    group_by(PersonId) %>%
    mutate(mean_lbs = mean(pounds, na.rm = TRUE), # calculate mean and sd . 
           sd_lbs =  sd(pounds, na.rm = TRUE)) |>
  mutate(distance_plausible =  .3*mean_lbs, # calculate plausible range (30%)
         plausible_lbs_ll = mean_lbs - .3*mean_lbs, # get plausible upper limit
         plausible_lbs_ul = mean_lbs + .3*mean_lbs) |>  # get plausible lower limit
    ungroup()

display_df(weight_data,con = con)

In [46]:
#Creates a column pounds2. If UOM_assumed2 == 'lbs' choose the weight in pounds, if the UOM_assumed2 value is 'kg' then choose the weight in kg.

weight_data <- weight_data |>
    mutate(lbsvalue_assume_lbs = ifelse(
             is.na(pounds) & 
             !is.na(Weight), 
             Weight, NA), # assume pounds if not NA and no pounds measure
           lbsvalue_assume_kg = ifelse(is.na(pounds) & 
             !is.na(Weight),
              Weight *2.205, NA), # assume kg if not NA and no pounds measure
           distance_lbs =  
             abs(lbsvalue_assume_lbs - mean_lbs), # distance between assumed lbs and mean lbs
           distance_kg = 
             abs(lbsvalue_assume_kg - mean_lbs), # distaance between assumed kg and mean lbs
          UOM_assumed2 = case_when(
             distance_lbs < distance_kg & 
             distance_lbs <= distance_plausible ~ "lbs", # when pounds distance is less than kg diff, and less = distance plausible then pounds
             distance_kg <  distance_lbs & 
             distance_kg<= distance_plausible ~ "kg",   # when kg distance is less than pounds diff, and less = distance plausible then kg
            TRUE ~ NA),
           pounds2 = case_when(
              UOM_assumed2 == "lbs" ~ lbsvalue_assume_lbs, # when assumed is pounds indicate pounds and use pounds value
              UOM_assumed2 == "kg" ~ lbsvalue_assume_kg, # when assumed is kg, indicate kg and use kg value calculated into pounds
              TRUE ~ pounds))

In [49]:
weight_data <- weight_data %>%
   mutate(pounds_final = coalesce(pounds, pounds2)) %>% # get final pounds
  select("PersonId" = PersonId,
  WeightDate,
  Weight,
  WeightUnit,
  pounds_final) # select and rename columns. 

display_df(weight_data)

In [20]:
# Merge final data with weight data 

sql <-
"
WITH temp as (
    SELECT  w.*, ABS(DATEDIFF(w.WeightDate, p.PanDt)) AS days
    FROM data p 
    INNER JOIN weight_data_sql w ON w.PersonId = p.PersonId
    WHERE w.WeightDate BETWEEN DATEADD(day, -360, p.PanDt) AND DATEADD(day, 360, p.PanDt)
),

temp2 as (
 SELECT *,  ROW_NUMBER() OVER (PARTITION BY PersonId ORDER BY days) AS rn
 FROM temp
)

SELECT PersonId,WeightDate,Weight,WeightUnit,pounds_final FROM temp2 WHERE rn=1
"
weight_alldata <- load_sql_table(con, snapshot, sql, view_name='weight_alldata',output_mode = "sparklyr") 

In [ ]:
vital_flag <- function(codes = "codes", lab_name = "name") {

  create_view(codes, "concept_codes")

  sql <- "
    WITH filtered AS (
      SELECT 
        l.PersonId,
        l.NormalizedValueNumeric,
        l.UnitOfMeasurement,
        l.EffectiveDateTime,
        '%s' as ConceptName
      FROM 
        bmi_data l
      INNER JOIN 
        concept_codes c ON l.CodeConceptId = c.ConceptId
    ),

    converted AS (
      SELECT 
        PersonId,
        EffectiveDateTime,

        -- Value conversion based on lab type
        CASE 
          WHEN ConceptName IN ('BMI') THEN
          
            
            CASE 
              WHEN UnitOfMeasurement = 'milligram per deciliter' THEN NormalizedValueNumeric
              WHEN UnitOfMeasurement = 'gram per liter' THEN NormalizedValueNumeric * 100   -- g/L to mg/dL
              WHEN UnitOfMeasurement = 'percent' THEN NormalizedValueNumeric / 100          -- to proportion, adjust logic if needed
              ELSE NULL
            END

          ELSE NULL
        END AS StandardValue,

        -- Standardized Unit Label
        CASE 
          WHEN ConceptName IN ('DirectBilirubin', 'TotalBilirubin') AND UnitOfMeasurement IN ('milligram per deciliter', 'gram per liter', 'percent') THEN 'mg/dL'
          ELSE NULL
        END AS Unit

      FROM 
        filtered
    ),

    latest_lab AS (
     SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY PersonId ORDER BY EffectiveDateTime DESC) AS rn
      FROM converted
        WHERE StandardValue IS NOT NULL AND Unit IS NOT NULL
        -- if Unit is NULL remove record
    )


    SELECT 
      PersonId, 
      ROUND(StandardValue, 3) AS %s,
      Unit AS %sUnit
    FROM latest_lab
    WHERE rn=1
  "
  
  sql1 <-  sprintf(sql,lab_name, lab_name, lab_name)

  tb <- load_sql_table(con, snapshot, query = sql1,output_mode = "sparklyr") %>% collect()
  return(tb)
}


In [39]:
#Merge bmi_temp, weight_data(with Personid, weight), imputed_bmi

cases_control_allvar_new <- cases_control_allvar_new %>% left_join(weight_alldata,by="PersonId")

### BMI Imputation

In [44]:
# ======================================
# STEP-BY-STEP BMI IMPUTATION USING MICE
# ======================================

library(mice)
#library(VIM)

# ------------------------------
# Step 1: Define Predictor Variables
# ------------------------------
comorbidity <- c("CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia",
                 "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes",
                 "Cancer", "Gastroesophageal_refluxdisease", "abdominal_pain", "Dyspnea", "Anemia", "FHOMND")

DemographicOth <- c("AgeAtDiagnosis", "Ethnicity", "Race", "Sex", "BMI","pounds_final")


predictors_for_imputation <- c(comorbidity, DemographicOth)

# ------------------------------
# Step 2: Subset Data for Imputation
# ------------------------------
imp_data <- cases_control_allvar_new %>% select(all_of(predictors_for_imputation))

# ------------------------------
# Step 4: Impute Using MICE
# ------------------------------
set.seed(123)
imp_result <- mice(imp_data, m = 5, method = 'pmm', maxit = 5, seed = 500)

# ------------------------------
# Step 5: Check Imputation Diagnostics
# ------------------------------
densityplot(imp_result$imp$BMI)  # Trace plot for BMI convergence
#display_df(imp_result$imp$BMI)

In [45]:
# ------------------------------
# Step 6: Extract Completed Dataset (first imputed dataset)
# ------------------------------
completed_data <- complete(imp_result, 1)
cases_control_allvar_new$BMI_imputed <- round(completed_data$BMI,2)

# Create a flag for imputed values
imputed_flag <- is.na(imp_data$BMI)

# Create data frame to compare
density_df <- data.frame(
  BMI = completed_data$BMI,
  Group = ifelse(imputed_flag, "Imputed", "Observed")
)

# Basic density plot
library(ggplot2)

ggplot(density_df, aes(x = BMI, fill = Group)) +
  geom_density(alpha = 0.5) +
  labs(
    title = "BMI Distribution: Observed vs Imputed",
    x = "BMI",
    y = "Density"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("Observed" = "blue", "Imputed" = "orange"))

In [ ]:
# Convert factors appropriately
# Already did

# ------------------------------
# Step 3: Visualize Missingness Before
# ------------------------------
aggr_plot_before <- aggr(imp_data, col=c('navyblue','red'),
                         numbers=TRUE, sortVars=TRUE,
                         labels=names(imp_data),
                         cex.axis=.7, gap=3,
                         ylab=c("Missing data","Pattern"))

In [46]:
cases_control_allvar_new <- cases_control_allvar_new %>%
  mutate(
    BMI_GROUP_impute = case_when(
      BMI_imputed < 18.5 ~ "Underweight",
      BMI_imputed >= 18.5 & BMI_imputed < 25 ~ "Normal",
      BMI_imputed >= 25 & BMI_imputed < 30 ~ "Overweight",
      BMI_imputed >= 30 & BMI_imputed < 35 ~ "Obese(Class I)",
      BMI_imputed >= 35 & BMI_imputed < 40 ~ "Obese(Class II)",
      BMI_imputed >= 40 ~ "Obese(Class III)",
      TRUE ~ "Unknown"  # Assigns NA if BMI is missing
    )
  )


In [22]:
# Read case_control_allvar_new data 
# Remove BMI, BMIDate, BMI_Group, BMI_imputed, BMI_GROUP_impute
# Merge my BMI data to this final table 

#cases_control_allvar_new <- cases_control_allvar_new %>% select(-BMI,-BMIUnit, - BMIDate, -BMI_Group, -BMI_imputed, -BMI_GROUP_impute)
#colnames(cases_control_allvar_new)
#bmi_temp <- collect(bmi_temp)
#cases_control_allvar_new <- cases_control_allvar_new %>% left_join(bmi_temp, by = "PersonId")

#***** Rename the BMI_GROUP original and create a new BMI_Group variable with Obese (II & III) in one category 

#cases_control_allvar_new <- cases_control_allvar_new %>% rename("BMI_Group_original" = "BMI_Group")

cases_control_allvar_new <- cases_control_allvar_new %>% mutate(BMI_Group = ifelse(BMI_Group_original == "Obese(Class III)", "Obese(Class II)", BMI_Group_original))

levels(as.factor(cases_control_allvar_new$BMI_Group))

In [ ]:
nrow(data)
length(unique(data$PersonId))
summary(data)
sum(is.na(data$BMI) & !is.na(data$pounds_final))

In [43]:
sum(is.na(cases_control_allvar_new$BMI) & !is.na(cases_control_allvar_new$pounds_final), na.rm=TRUE)

In [ ]:
# Create a Raw BMI data

sql <-
"
   SELECT  t.PersonId,  t.EffectiveDateTime AS BMIDate, t.NormalizedValueNumeric AS BMI
     FROM SearchResult_defTrBodyMassIndexBmi t INNER JOIN data p ON t.PersonId = p.PersonId
   INNER JOIN temp1 t ON t.PersonId = p.PersonId

"
bmi_raw <- load_sql_table(con, snapshot, sql, view_name='bmi_temp',output_mode = "sparklyr") 

In [ ]:
# old logic
# Take the latest value of BMI within PanDt - 1 year to PanDt 
sql1 <- "

WITH BMI_Ranked AS (
    SELECT 
        p.PersonId, p.SEX, round(b.NormalizedValueNumeric,2) as BMI, b.EffectiveDateTime AS BMIDate,
        ROW_NUMBER() OVER (
            PARTITION BY p.PersonId 
            ORDER BY b.EffectiveDateTime DESC
        ) AS rn
    FROM data p
    LEFT JOIN SearchResult_defTrBodyMassIndexBmi b
        ON p.PersonId = b.PersonId
    WHERE (b.EffectiveDateTime BETWEEN DATEADD(day, -360, p.PanDt) AND DATEADD(day, 30, p.PanDt)) 
    AND b.NormalizedValueNumeric IS  NOT NULL
    AND b.NormalizedValueNumeric >= 10 AND b.NormalizedValueNumeric <= 90 AND 
    year(b.EffectiveDateTime) >= 2018
   ),

BMI_Ranked1 AS(
    SELECT *
    FROM BMI_Ranked
    WHERE rn = 1
  )

SELECT PersonId,BMI, BMIDate,
  CASE 
    WHEN BMI < 18.5 THEN 'Underweight'
    WHEN BMI >= 18.5 AND BMI < 25 THEN 'Normal'
    WHEN BMI >= 25 AND BMI < 30 THEN 'Overweight'
    WHEN BMI >= 30 AND BMI < 35 THEN 'Obese(Class I)'
    WHEN BMI >= 35 AND BMI < 40 THEN 'Obese(Class II)' 
    WHEN BMI >= 40 THEN 'Obese(Class III)'
  END AS BMI_Group
  FROM BMI_Ranked1

-- if median bmi is chosen instead of latest
-- PERCENTILE_APPROX(b.NormalizedValueNumeric, 0.5) AS BMI 

"
pan_bmi <- load_sql_table(con, snapshot, sql1, view_name='pan_bmi',output_mode = "sparklyr") %>% collect()
#display_df(pan_bmi)

In [53]:
display_df(cases_control_allvar_new[is.na(cases_control_allvar_new$BMI),])

In [47]:
colnames(cases_control_allvar_new)